In [3]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as T
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda")

batch_size_train = 64
batch_size_test = 1000
lr = 1
momentum = 0.7

tasks = 10

In [2]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.ff1 = nn.Linear(28*28, 512)
        self.ff2 = nn.Linear(512, 10)
        self.log_softmax = F.log_softmax

    def forward(self,x):
        x = F.relu(self.ff1(x))
        x = self.log_softmax(self.ff2(x), dim=1)
        return x

def train(model, device, train_loader, optimizer):
    model.train()
    correct = 0
    for batch_i, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        #For every batch make gradient zero
        optimizer.zero_grad()
        output = model(data)
        #Compute the negative log likelihood
        loss = F.nll_loss(output, target)
        #Backprobagate the loss
        loss.backward()
        optimizer.step()

        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

    train_acc = 100. * correct / len(train_loader.dataset)
    return train_acc

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_acc = 100. * correct / len(test_loader.dataset)
    return test_loss, test_acc

In [9]:
def first_test(model, device):
    transform = T.Compose([
        T.ToTensor(),
        T.Normalize((0.1307),(0.3081)),
        T.Lambda(lambda x: torch.flatten(x))
    ])
    train_df = datasets.MNIST('../data', train=True, download=True, transform=transform)
    test_df = datasets.MNIST('../data', train=False, download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_df, batch_size=batch_size_train)
    test_loader = torch.utils.data.DataLoader(test_df, batch_size=batch_size_test)
    optimizer = optim.Adadelta(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=momentum)

    for epoch in range(10):
        train_acc = train(model, device, train_loader, optimizer)
        test_loss, test_acc = test(model, device, test_loader)
        scheduler.step()
        print(f'Epoch {epoch+1} train/test accuracy: {train_acc:.2f}/{test_acc:.2f}')

model = NN().to(device)
first_test(model, device)

Epoch 1 train/test accuracy: 94.48/96.91
Epoch 2 train/test accuracy: 98.25/97.60
Epoch 3 train/test accuracy: 99.11/97.93
Epoch 4 train/test accuracy: 99.52/98.05
Epoch 5 train/test accuracy: 99.75/98.18
Epoch 6 train/test accuracy: 99.82/98.23
Epoch 7 train/test accuracy: 99.88/98.30
Epoch 8 train/test accuracy: 99.89/98.37
Epoch 9 train/test accuracy: 99.92/98.41
Epoch 10 train/test accuracy: 99.93/98.40


In [13]:
def generate_task(random_seed):
    rnd = np.random.RandomState(random_seed)
    idx_permute = torch.from_numpy(rnd.permutation(28*28))

    transform = T.Compose([
        T.ToTensor(),
        T.Normalize((0,.1307),(0.3081)),
        T.Lambda(lambda x: torch.flatten(x)[idx_permute])
    ])

    train_df = datasets.MNIST('../data', train=True, download=True, transform=transform)
    test_df = datasets.MNIST('../data', train=False, download=True, transform=transform)

    return train_df, test_df

def test_all(model, device, tasks):
    accs = []
    for j in range(len(tasks)):
        [train_df, test_df] = tasks[j]
        test_loader = torch.utils.data.DataLoader(test_df, batch_size=test_bs)
        tets_loss, test_acc = test(model, device, test_loader)
        accs.append(test_acc)
        print(f'Test on task {j+1}: loss {test_loss:.4f}, accuracy {test_acc:.2f}')
    return accs

tasks = [generate_task(random_seed=i) for i in range(len(tasks))]
# plt.imshow(tasks[0][0][0][0].reshape(28,28))


In [16]:
len(tasks)

10

In [26]:
tasks[0][0].help() #Task 0, training set

AttributeError: 'MNIST' object has no attribute 'help'